# Aplicando Transformers en Clasificación de Texto.

**Investigadores**: <br>
  Dr. Ramón Zatarain Cabada<br>
  Dra. María Lucía Barrón Estrada<br>
  M.C. Víctor Manuel Bátiz Beltrán

**Corpus**: EduSERE

**Referencias**:

- Barrón Estrada, M. L., Zatarain Cabada, R., Oramas Bustillos, R., & Graff, M. (2020). Opinion mining and emotion recognition applied to learning environments. Expert Systems with Applications, 150, 113265. https://doi.org/10.1016/j.eswa.2020.113265

- Zatarain Cabada, R., Barrón Estrada, M. L., Bátiz Beltrán, V. M. (2023). Advanced Applications of Generative AI and Natural Language Processing Models (Chapter 15). Deep Learning Approaches for Affective Computing in Text (pages 306-339). DOI: 10.4018/979-8-3693-0502-7.ch015.


### Descripción general

Usaremos el dataset EduSERE.

El corpus se divide en tres emociones orientadas al aprendizaje: frustrado, aburrido y comprometido (enganchado). El corpus tiene 3245 textos clasificados como frustrado, 3239 textos clasificados como aburrido y 5600 textos clasificados como comprometido (enganchado). Los textos del corpus están en español.

### Primeros pasos
Instalamos e importamos las bibliotecas a utilizar.

In [ ]:
!pip install emoji

In [ ]:
!pip install gensim

In [ ]:
import re
#import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import SnowballStemmer
import unicodedata
from collections import Counter
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
#import seaborn as sns
#from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import emoji
import keras
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
print('Listo')

## 1. Cargamos el dataset

### Descargando el corpus.

La primera celda de código fue necesaria para poder usar el mode GPU, ya que sin ello marcaba error de encoding.

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
def corpus_download(path, url):
  !wget --no-check-certificate \
     {url} \
     -O {path}

In [ ]:
corpus_download("EduSere.csv","https://person-app-itc.web.app/corpus/EduSere.csv")

In [ ]:
data = pd.read_csv("EduSere.csv")

### Exploración de los datos

In [ ]:
data.head()

In [ ]:
len(data)

Cambiamos las etiquetas a representación numérica 0 = frustrado (frustrated), 1 = aburrido (bored) y 2 = comprometido (engaged).


In [ ]:
data['Label'] = data['Label'].replace({'frustrado':0, 'aburrido':1, 'comprometido':2})

## 2. Limpieza de datos (Data cleaning)

In [ ]:
data.head()

In [ ]:
data['Label'].value_counts()

Como podemos observar tenemos un desbalance entre las clases. Tomaremos 3000 registros por cada clase para tenerlas balanceadas.

In [ ]:
#Undersampling
frustrated = data[data['Label']==0]
bored = data[data['Label']==1]
engaged = data[data['Label']==2]

frustrated = frustrated.sample(n=3000, random_state=1)
bored = bored.sample(n=3000, random_state=1)
engaged = engaged.sample(n=3000, random_state=1)

data = pd.concat([frustrated, bored, engaged], axis=0)


In [ ]:
data.head()

In [ ]:
print(data.dtypes)


In [ ]:
#Check if we have null fields
data.isnull().sum()

In [ ]:
#In case we have null texts.
data["Text"].fillna("Sin texto", inplace = True)

### A continuación realizaremos los siguientes pasos:

1. Separar el texto en Tokens
2. Convertir palabras a minúsculas
3. Expandir contracciones
4. Remover urls, correos, saltos de línea
5. Eliminar caracteres repetidos
6. Eliminar nuevas líneas y pestañas
7. Remover saltos de línea
8. Remover comillas simples
9. Eliminar comas " , "
10. Remover números
11. Remover Caracteres no alfanuméricos
12. Eliminar guiones entre palabras
13. Eliminar los guiones dobles y triples
14. Eliminar espacios en blanco (al principio, final y espacios dobles)
15. Eleminar stop words
16. Realizar stemming/Lematizacion  
17. Remover signos de puntuación
18. Destokenizar


In [ ]:
def process_text(sentence, norm_user = True, norm_hashtag = True, separate_characters = True):
    # Convert instance to string
    sentence = str(sentence)

    # All text to lowecase
    sentence = sentence.lower()

    # Normalize users and url
    if norm_user == True:
        sentence = re.sub(r'\@\w+','@usuario', sentence)
    if norm_hashtag == True:
        sentence = re.sub(r"http\S+|www\S+|https\S+", 'url', sentence, flags=re.MULTILINE)

    # Separate special characters
    if separate_characters == True:
        sentence = re.sub(r":", " : ", sentence)
        sentence = re.sub(r",", " , ", sentence)
        sentence = re.sub(r"\.", " . ", sentence)
        sentence = re.sub(r"!", " ! ", sentence)
        sentence = re.sub(r"¡", " ¡ ", sentence)
        sentence = re.sub(r"“", " “ ", sentence)
        sentence = re.sub(r"'", " ' ", sentence)
        sentence = re.sub(r"”", " ” ", sentence)
        sentence = re.sub(r"\(", " ( ", sentence)
        sentence = re.sub(r"\)", " ) ", sentence)
        sentence = re.sub(r"\?", " ? ", sentence)
        sentence = re.sub(r"\¿", " ¿ ", sentence)

    # Substituting multiple spaces with single space
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)
    # emojis to text
    sentence = emoji.demojize(sentence)

    return sentence

In [ ]:
clean_data = data.copy()
clean_data['Text'] = clean_data['Text'].apply(process_text)

In [ ]:
clean_data.head()

## Eliminamos las palabras que no aportan valor (stopwords).

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('stopwords')
print(stopwords.words('spanish'))

In [ ]:
stop_words = set(stopwords.words('spanish'))

In [ ]:
def remove_stopwords(text):
  word_tokens = word_tokenize(text)
  no_stopwords = [word for word in word_tokens if not word in stop_words]
  return " ".join(no_stopwords)

In [ ]:
remove_stopwords('el que tiene tienda la debe atender')

In [ ]:
clean_data['Text'] = clean_data['Text'].apply(remove_stopwords)

## Lematización

In [ ]:
#https://spacy.io/models/es
#We'll use Spacy for Lematization
!python -m spacy download es_core_news_sm

In [ ]:
import spacy
import es_core_news_sm
nlp = es_core_news_sm.load()

In [ ]:
def lematize(text):
    doc = nlp(text)
    lemms = []
    for token in doc:
        lemms.append(token.lemma_)
    return " ".join(lemms)

In [ ]:
lematize('yo soy muy feliz con mi familia')

In [ ]:
clean_data['Text'] = clean_data['Text'].apply(lematize)

## Eliminamos signos de puntuación y acentos (Punctuation Cleaning).



In [ ]:
def cleaning_punct(text):
  token_list = gensim.utils.simple_preprocess(str(text), deacc=True)  # deacc=True remueve puntuación
  return " ".join(token_list)

In [ ]:
cleaning_punct('mi méxico querido qué fantástico')

In [ ]:
clean_data['Text'] = clean_data['Text'].apply(cleaning_punct)

### Codificación de las etiquetas

Como el conjunto de datos es categórico, necesitamos convertir las etiquetas de personalidad de Neutral, No y Sí a un tipo float que nuestro modelo pueda entender. Para lograr esta tarea, implementaremos el método to_categorical de Keras.

## Construcción del modelo

### Transformers

Pasos iniciales

In [ ]:
!pip install transformers==4.24.0
!pip install simpletransformers==0.63.11

In [ ]:
#!pip install transformers
#!pip install simpletransformers

In [ ]:
pip show simpletransformers  #Para ver la versión instalada

### Cargamos los modelos preentrenados.

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
import logging # Import the logging module

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
clean_data2 = clean_data.copy()
clean_data2.rename(columns = {'Text':'text','Label':'labels'}, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(clean_data2, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 1,  #Usaremos una época por temas de tiempo
             "overwrite_output_dir": True}

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-uncased',
    num_labels=3,
    args=train_args
)

In [ ]:
# Train the model
model.train_model(train_df)

In [ ]:
# Cargamos las métricas
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# Evaluamos el modelo
result, model_outputs, wrong_predictions = model.eval_model(
    test_df,
    f1=lambda labels, preds: f1_score(labels, preds, average='weighted'),  # Use weighted average for F1-score
    acc=accuracy_score,  # Accuracy doesn't need averaging for multi-class
    rc=lambda labels, preds: recall_score(labels, preds, average='weighted'),  # Use weighted average for recall
    pcs=lambda labels, preds: precision_score(labels, preds, average='weighted')  # Use weighted average for precision
)

In [ ]:
print(f" Exactitud (Accuracy): {result['acc']}")
print(f" F1-Score: {result['f1']}")
print(f" Recall: {result['rc']}")
print(f" Precisión: {result['pcs']}")

### Probando el modelo

In [ ]:
#from sklearn.metrics import recall_score
from sklearn import metrics

In [ ]:
#Recordemos nuestras clases frustrado':0, 'aburrido':1, 'comprometido':2
clases = ['Frustrado','Aburrido','Comprometido']

In [ ]:
# Vamos a usar un diccionario para crear el dataset de prueba
# Frases:
# 0-Que tristeza estar en este taller, siento que pierdo mi tiempo
# 1-Que taller tan tedioso, no me motiva a nada
# 2-Este taller esta genial, los instructores explican muy bien
datos = {
    'text': ['Que tristeza estar en este taller, siento que pierdo mi tiempo'],
    'labels': [0]
}

# Crear un DataFrame a partir del diccionario
df = pd.DataFrame(datos)

In [ ]:
df.head()

In [ ]:
test = df['text'].to_numpy().tolist()
y = df['labels'].to_numpy().tolist()
print(test[0])
print(y[0])
print(len(test))
print(len(y))

In [ ]:
predictions_test = model.predict(test)

In [ ]:
#Accediendo la clase elegida por el modelo
print(clases[predictions_test[0][0]])

In [ ]:
#Usando el vector de probabilidades
print(clases[np.argmax(predictions_test[1])])

In [ ]:
test_recall = metrics.recall_score(y, predictions_test[0], average='macro')
test_f1 = metrics.f1_score(y, predictions_test[0], average='macro')
test_precision = metrics.precision_score(y, predictions_test[0], average='macro')
test_accuracy = metrics.accuracy_score(y, predictions_test[0])

In [ ]:
print("Metrics results:")
print(f"Accuracy: {test_accuracy}")
print(f"F1: {test_f1}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")